# Iterables, Iterators, and Generators in Python

Until recently, I've always been somewhat confused about the exact meanings of and relationships among
the three terms *iterable*, *iterator*, and *generator*.
So I'm making this notebook as a resource for clarifying this topic, primarily for my own benefit, but I hope it can help others too.

First I'll give a short overview of the terms, their meanings, and their relationships to one another.
Then I'll dig deeper into each of the three terms.
Finally, I'll show some cool things you can do with iterators.

## Overview

#### Iterables
An iter**able** is any object you can call `iter` on to produce an iterator.
In plainer terms, it's pretty much any object you can loop over with a `for ... in ...` loop.

#### Iterators
An iter**ator** is an object that you can repeatedly call `next` on to produce values one at a time,
perhaps until it runs out of values and raises a `StopIteration` exception.
The gist is that iterators are streams of data: they produce values one at time, and they don't store their previous values.
By convention, every iterator is also an iterable, and calling `iter` on an iterator returns the same iterator you started with.

#### Generators
As we'll see below, implementing our own iterators from scratch can be a little more verbose than we might like.
Generators are a special case of iterators, but they are meant to be friendlier to write. That's all.
There are two kinds of generators: generator functions and generator expressions.
Generator functions use function definition syntax to create iterators,
while generator expressions use a list comprehension style syntax to create iterators.
If you need to write your own iterator, you'll probably use a generator for convenience.

#### Recap
Calling `iter` on an iterable produces an iterator. Every iterator is itself an iterable,
because calling `iter` on an iterator returns that same iterator (`iter` is idempotent).
Generators are iterators, and thus they are also iterables. 
Iterators are streams of data.
You can access the next value of an iterator with `next`,
but most of the time you'll use a loop or some other higher-level construct.

## Iterables

As I just mentioned, an iterable is anything that produces an iterator when you call `iter` on it.
I also said that it is anything you can loop over with `for ... in ...`.
I'll show exactly how for loops work in the iterator section.
After we've talked about iterators, I can show how to build your own iterables and iterators.
Right now, I'll show some examples of iterables.
Pretty much any container (list, set, dict) is iterable.

Here we create a list `x` and then call `iter` on it.
We get back a `list_iterator`.
This shows that lists are iterable, as we already knew since we loop over them all the time.

In [28]:
x = [1, 2, 3]
iter(x)

Below, we do a similar thing with a set, a dictionary, the items of a dictionary, and a string.
`iter` always produces some kind iterator.
This also shows that there isn't an iterator type like there is, say, a single `str` type.
Rather, anything that acts like an iterator is one.
Similarly, there isn't an iterable type; anything that acts like an iterable is one.

In [29]:
x = {1, 2, 3}
iter(x)

In [30]:
x = {1:1, 2:2, 3:3}
iter(x)

In [31]:
iter(x.items())

In [32]:
x = "abc"
iter(x)

We also see that iterators are themselves iterable.
And that calling `iter` does nothing, as promised.

In [34]:
x = [1, 2, 3]
y = iter(x)
z = iter(y)
y is z

True

## Iterators

Okay, here's where things get a little more interesting.
Let's make some iterators and show how to get data from them using `next`.


Below, I made a small list, `x`, and called `iter` on it to produce a `list_iterator`, `y`.
Calling `next` on `y` repeatedly returns the values from `x`, in order, and then raises a `StopIteration` exception.

In [35]:
x = [1, 2, 3]
y = iter(x)

In [36]:
next(y)

1

In [37]:
next(y)

2

In [39]:
next(y)

3

In [41]:
next(y)

StopIteration: 

So, if we wanted to loop over all the values from an iterator and print them, we could do something like this:

In [42]:
x = [1, 2, 3]
y = iter(x)
while True:
    try:
        print(next(y))
    except StopIteration:
        break

1
2
3


We see that the code above prints `1, 2, 3` and then halts successfully.

In fact, this is basically how a for loop works!
When we write `for i in x`, where `x` is an iterable, a few things happen:
1. Python calls `iter` on `x` to produce an iterator.
2. Then Python repeatedly calls `next` on that iterator to produce values one by one.
3. Finally, Python catches any `StopIteration` exception that is raised,
and uses that as the signal that the iterator is exhausted and the loop should end.


So, whenever you write a basic for loop you're utilizing iterables, iterators, and exception handling!
This also makes it clear why it's convenient to be able to call `iter` on an iterator.
This way we can loop directly over an iterator.

As one final note for this section, the step where we call `iter` on an iterable is important, since iterables do not have to be iterators!

In [119]:
next([1, 2, 3])

TypeError: 'list' object is not an iterator

## Writing Your Own Iterables and Iterators

So far I've just shown examples of iterables and iterators arising from the basic container types in Python.
Now I'll show how create your own iterables and iterators.

As I've said so many times now, an iterable is something that produces an iterator when `iter` is called on it.
So if we're writing a class, and we want it to be iterable, we just need to make sure `iter` works the way we want on our class.
There are two ways to do this.
The most direct way is to implement the `__iter__` method inside the class.
This directly corresponds to the `iter` function.


The other way is to implement the `__getitem__` method.
`x.__getitem__(i)` directly corresponds to `x[i]`.
If we've implemented `__getitem__`, then calling `iter` will try to create an iterator by feeding in the indices
`0, 1, 2, ...` to `__getitem__` and will raise a `StopIteration` exception when `__getitem__` raises an `IndexError`.


Of course, we can't really make an iterable if we don't know how to make an iterator.
So let's discuss that quickly.
Just like `iter` and `__iter__`, we can define how the `next` function works for a class by implementing the `__next__` method.
We just need to make sure that `__next__` raises `StopIteration` when (and if) we want the iterator to stop.
That's really all there is to it.


Okay, I know that was a lot, but now we're ready to go!


First, let's produce a simple iterable that when iterated over produces the numbers `0` to `n - 1`, like `range`.

In [58]:
class RangeIterable:
    """Iterable that works like range."""
    
    def __init__(self, n):
        self.n = n
    
    def __iter__(self):
        return RangeIterator(self.n)
    
class RangeIterator:
    """Iterator that produces 0 to n - 1"""
    
    def __init__(self, n):
        self.n = n
        self.state = 0
        
    def __next__(self):
        if self.state < self.n:
            value = self.state
            self.state += 1
            return value
        else:
            raise StopIteration
        
for i in RangeIterable(5):
    print(i)

0
1
2
3
4


Above, we have the two classes we need to make this work:
an iterable class and an iterator class.
The iterable class implements `__iter__`, and the iterator class implements `__next__`.
We can see that the for loop works exactly as expected!
There, that wasn't so bad, was it?

If we want to adhere to best practices, our iterator should also implement `__iter__`.
So let's make that change now.

First we show that `RangeIterator` is currently not iterable.

In [59]:
for i in RangeIterator(5):
    print(i)

TypeError: 'RangeIterator' object is not iterable

Now we fix that and show that we can loop over it.

In [60]:
class RangeIterator:
    """Iterator that produces 0 to n - 1"""
    
    def __init__(self, n):
        self.n = n
        self.state = 0
        
    def __next__(self):
        if self.state < self.n:
            value = self.state
            self.state += 1
            return value
        else:
            raise StopIteration
            
    def __iter__(self):
        return self
    
for i in RangeIterator(5):
    print(i)

0
1
2
3
4


Now, since we can loop directly over the `RangeIterator`, you might be wondering why we even need the `RangeIterable` class.
The answer is, we don't need it!
But there is something crucial to understand here.
One `RangeIterable` can produce an unlimited supply of `RangeIterator`s, since everytime we call `iter` we get a new iterator.
This means we can repeatedly loop over it.
A single `RangeIterator` is only good for one loop before it's spent.
Calling `iter` on an iterator does not make a new iterator.
Observe.

In [55]:
x = RangeIterable(5)

print("Loop 1:")
for i in x:
    print(i)
    
print("Loop 2:")
for i in x:
    print(i)

Loop 1:
0
1
2
3
4
Loop 2:
0
1
2
3
4


In [56]:
x = RangeIterator(5)

print("Loop 1:")
for i in x:
    print(i)
    
print("Loop 2:")
for i in x:
    print(i)

Loop 1:
0
1
2
3
4
Loop 2:


The `RangeIterator` doesn't produce any values in the second loop.
This is something you have to be careful with when using iterators.
So, while it is generally true that we don't need to make a separate iterable class to accompany our iterator,
it can be nice if you need to loop over the same object more than once.


Before we move on to generators, I want to quickly show how implementing `__getitem__` affects `iter`.

Any class that implements `__getitem__` in a 0-based sequence style is automatically iterable.
Below, I've re-implemented the `RangeIterable` class using `__getitem__` instead of `__iter__`.
In this particular case, this implementation ends up being very simple.
Now Python handles making the iterator for us by feeding in indices `0, 1, 2, ...`  to `__getitem__` and stoping when `__getitem__` raises an `IndexError`.

In [64]:
class RangeIterable:
    
    def __init__(self, n):
        self.n = n
        
    def __getitem__(self, i):
        if i < self.n:
            return i
        else:
            raise IndexError
            
for i in RangeIterable(5):
    print(i)

0
1
2
3
4


Note that implementing `__iter__` overrides this relationship between `__getitem__` and `iter` as this next code snippet shows.

In [65]:
class RangeIterable:
    
    def __init__(self, n):
        self.n = n
        
    def __getitem__(self, i):
        if i < self.n:
            return i
        else:
            raise IndexError

    def __iter__(self):
        return iter("abc")
    
for i in RangeIterable(5):
    print(i)

a
b
c


## Generators

Okay, so now we've seen how to make any iterable or iterator our hearts desire.
But the approaches we've seen so far tend to be somewhat verbose.
Wouldn't it be nice if there were a concise, lightweight way to define new iterators?
Generators are the solution we want!

There are actually two fairly unrelated things that are both called generators.
Both of them are much more concise than class-based iterators.
The first kind of generator we'll cover is called a generator function.
After that, we'll look at generator expressions.

#### Generator Functions
A generator function is just like an ordinary function, except we use the `yield` keyword where we might otherwise use `return` or append to a list.
Generator functions are very concise compared to making a class-based iterator.
The savings in typing should be obvious, so instead I'll focus on the differences between generator functions and ordinary functions.

For example, compare the following two functions. The first creates a list of all the squares from `0` to `(n - 1) ** 2`.
The second is a generator function that returns an iterator that produces the same values.
Besides avoiding a little bit of boilerplate, the generator function has the advantages that 
we don't need to construct the whole list of squares before beginning to iterate, nor do we need to store all the squares in memory.
But as we mentioned previously, the iterator can only be iterated over once, whereas the list returned from the first function can be iterated
over as many times as we please.

In [88]:
def squares_lst(n):
    lst = []
    for i in range(n):
        lst.append(i ** 2)
    return lst

def squares_gen(n):
    for i in range(n):
        yield i ** 2
        
x = squares_lst(3)
y = squares_gen(3)


# sum is 0 + 1 + 4 = 5
# list can be summed more than once
print("list:")
print(sum(x))
print(sum(x))

# generator is an iterator and is gone after being summed once
print("generator:")
print(sum(y))
print(sum(y))

list:
5
5
generator:
5
0


Although it may be intuitively obvious what is happening, let's take a moment to discuss how the generator functions work.
When a generator function is called, it returns an iterator.
When we ask for a value from that iterator, we enter the function body and proceed until the first `yield` statement.
Whatever is `yield`ed is the first value of that iterator.
With an ordinary function, the environment inside the function body is destroyed upon `return`ing a value.
But with a generator function, we save the environment for later use.
Next time we need a value from the iterator, we go back to the same environment and proceed until we reach another `yield` statement.
This continues until the generator function `return`s any value (it doesn't matter what value is returned),
if ever, and then the iterator raises a `StopIteration` exception.
In the generator function example above, there is an implicit `return None` after the end of the for loop,
since every function without a `return` statement implicitly has `return None` at the very end of the function body.

In short, generator functions run until a `yield` statment and then save their state and pause until we ask for the next value.
This continues until a `return` statement. 

Generator functions are a great way write concise iterators.
They are also a nice way to save memory and possibly increase speed as compared to an ordinary function that would build up one big data structure.
Finally, generator functions (and iterators more generally) allow you to work with infinite streams of data,
where it would be impossible to first load all the data before processing.


#### Generator Expressions
The second kind of generator is called a generator expression.
A generator expression has the exact same syntax as a list comprehension, except the outer `[` and `]` are replaced by parentheses.
Instead of producing a list, the generator expression produces an iterator.
Though not as widely applicable, generator expressions tend to be even more concise than generator functions.
The main benefits over list comprehensions are much less memory usage, and possibly some speed enhancements.
The downside, once again, is that a generator expression can only be iterated over once before being exhausted.
See the example below.

In [87]:
lst = [x ** 2 for x in range(10)]
gen = (x ** 2 for x in range(10))

# the list can be summed over more than once
print("list:")
print(sum(lst))
print(sum(lst))

# the generator can only be summed over once
# then the empty sum is 0
print("generator:")
print(sum(gen))
print(sum(gen))

list:
285
285
generator:
285
0


Of course, if you're immediately going to consume the values in a for loop or `sum` or other function,
then there's usually no reason to prefer the list comprehension over the generator expression.
And, if you're placing the generator expression directly inside the parentheses of a function call,
you can omit one set of parentheses, which looks quite nice.

In [92]:
# generator expression inside function call
sum((x ** 2 for x in range(10)))

285

In [91]:
# works without two sets of parens
sum(x ** 2 for x in range(10))

285

## Cool Iterators

As promised, I'm going to close by showing off some cool things you can do with iterators.
First, let's write as basic Fibonacci number iterator, since it's a programming classic.

In [107]:
def fib(n):
    """yields first n fibonacci numbers"""
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b
        
list(fib(10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

Now, as a neat trick, let's create an infinite fibonacci number generator.
Now we can get more and more fibonacci numbers whenever we ask for them, without having to decide in advance how many we need, which I think is conceptually nice.
We'll need an extra function for actually extracting the values now.

In [112]:
def fib():
    """yields all the fibonacci numbers"""
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b
        
def head(iterator, n):
    i = 0
    while i < n:
        yield next(iterator)
        i += 1
        
print(list(head(fib(), 10)))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


We can also make recursive, infinite generators.
As always, deep recursion is not such a good idea in Python, since we'll eventually run out of recursion depth.
But this stuff is pretty cool and mind-bending, so I wanted to show it off.
If you want a language that handles this well, try Haskell.

In [113]:
def ones():
    """yields constant sequence of ones"""
    yield 1
    for elt in ones():
        yield elt
        
list(head(ones(), 10))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Above, we define an infinite sequence of 1s by saying that the first element is 1, and then the rest of the elements are the very sequence we're defining! Trippy, right?

Also, Python has a nice shorcut for yielding from another iterator: `yield from`

In [115]:
def ones():
    """yields constant sequence of ones"""
    yield 1
    yield from ones()
    
list(head(ones(), 10))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Let's make a less trivial recursive generator.

Below we make a natural number generator using the fact that the first natural number is zero,
and the rest of the natural numbers are just the natural numbers + 1.
I use a generator expression with `yield from` to streamline the code.

In [116]:
def nats():
    """yields the natural numbers 0, 1, 2, 3, ..."""
    yield 0
    yield from (i + 1 for i in nats())
    
list(head(nats(), 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Now as our final trick, let's make a recursive infinite fibonacci number generator.

The implementation is based on the following description of the fibonacci numbers:
- The first two fibonacci numbers are 0 and 1.
- Then, the fibonacci numbers from the third on are the sum of the fibonacci numbers and the tail of the fibonacci numbers,
where the tail of a sequence is just the sequence with the first term dropped.

In other words, element-wise addition of the fibonacci numbers and the the fibonacci numbers with the first element omitted
gives the fibonacci numbers with the first two elements omitted.

```
   0, 1, 1, 2, 3, 5, 8,  ...
+  1, 1, 2, 3, 5, 8, 13, ...
____________________________
=  1, 2, 3, 5, 8, 13, 21, ...
```


In [117]:
def tail(iterator):
    next(iterator)
    yield from iterator
    
def fib():
    yield 0
    yield 1
    yield from (a + b for a, b in zip(fib(), tail(fib())))
    
list(head(fib(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

That's some beautiful and terse code, in my opinion. 

Unfortunately it is also some very slow code. Every time we reach the `yield from` line we spin up two more `fib()` generators. It has the same terrible exponential time complexity as a naive fibonacci implementation. 

Fortunately, we can do better. We don't need two whole copies of `fib()` for the `yield from` step. If we could somehow couple the two generators so that they both get their values from the same source, but just with one of them one step behind, then we could avoid the exponential blow up. The `tee` function from `itertools` does exactly this. It creates copies of an iterator which then all share the resources of the original iterator. Values produced by the iterator but not yet consumed by all the copies wait in a queue. Once every copy has advanced beyond a value, it is no longer stored. `tee` is perfect for our use case, since our copies will only ever be one value apart. If, however, we made copies of an iterator with `tee` and then one advanced through all the values, while the other hadn't advanced at all, then this would be space equivalent to just storing the iterator in a list to begin with. Thus, `tee` only helps when we expect there to be no badly 'lagging' copies.

As a final modification, I'll also swap out the `zip` generator expression for a `map`, just to show another way to achieve the same thing and because it ends up being a litle bit shorter. By the way, `zip` and `map` both return iterators, which is the only way they could possibly work on these infinite iterators. I used to be annoyed that `zip` and `map` wouldn't just give me a list, but now I appreciate the power of iterators.

Behold the terse elegance of our fast, infinite, recursive fibonacci generator!

In [118]:
from operator import add

def fib():
    yield 0
    yield 1
    fib1, fib2 = it.tee(fib())
    yield from map(add, fib1, tail(fib2))
    
list(head(fib(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

As a final note, this recursive fibonacci generator is still not as practical as the earlier iterative one, since this recursive one will evetually run into recursion depth problems.

I hope this notebook has helped to resolve any confusion you might feel about iterables, iterators, and generators.